In [1]:
# import packages
import pandas as pd
import numpy as np

# inline plotting
% matplotlib inline

# block scientific notation; round to numeric variables to  two decimal places
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# set display options
pd.set_option('display.max_columns', 400)
pd.set_option('display.max_rows',400)

In [2]:
# read in train, test, and sampel submission files
train_df = pd.read_json('train.json')
test_df = pd.read_json('test.json')
#sample_sub_df = pd.read_csv('sample_submission_2.csv')
train_df.shape, test_df.shape

((49352, 15), (74659, 14))

In [3]:
full_df = pd.concat([train_df,test_df], axis=0, ignore_index=True).copy()
full_df.shape

(124011, 15)

In [4]:
# convert to datetime data type
# break data into individual columns

from datetime import datetime

full_df['created'] = pd.to_datetime(full_df['created'], format='%Y-%m-%d %H:%M:%S')
full_df['year'] = full_df['created'].dt.year
full_df['month'] = full_df['created'].dt.month
full_df['day'] = full_df['created'].dt.day
full_df['hour'] = full_df['created'].dt.hour
full_df['weekday'] = full_df['created'].dt.weekday

In [5]:
# links to photos are stored in array
# taking the length of the array provides the count of photos per observation

full_df['photos_count'] = full_df.photos.apply(len)

In [6]:
# feature names are stored in array
# taking the length of the array provides the count of features per observation

full_df['features_count'] = full_df.features.apply(len)

In [7]:
# price divided by bedrooms
full_df['price_per_bed'] = full_df.apply(lambda x: x.price if x.bedrooms == 0\
                                         else x.price / x.bedrooms, axis=1)

In [8]:
# price divided by (bedrooms + bathrooms)
full_df['price_per_bedbath'] = full_df.apply(lambda x: x.price if x.bedrooms + x.bathrooms == 0\
                                             else x.price / (x.bedrooms + x.bathrooms), axis=1)

In [9]:
# dummy variable for elevator value in 'features' column
full_df['Elevator'] = 0
elevator_list = ['elevator', 'elevators', 'Elevators', 'Elevator']
for row in range(0,len(full_df)):
    for feature in full_df.iloc[row]['features']:               
        if feature in elevator_list:
            full_df.ix[row,'Elevator'] = 1

In [10]:
# dummy variable for doorman value in 'features' column
full_df['Doorman'] = 0
door_list = ['Doorman','doorman','door man', 'Door man', 'Door Man']
for row in range(0,len(full_df)):
    for feature in full_df.iloc[row]['features']:               
        if feature in door_list:
            full_df.ix[row,'Doorman'] = 1

In [11]:
# dummy variable for hardwood value in 'features' column
full_df['Hardwood'] = 0
hardwood_list = ['Hardwood','hardwood','hard wood', 'Hard wood', 'Hard Wood']
for row in range(0,len(full_df)):
    for feature in full_df.iloc[row]['features']:               
        if feature in hardwood_list:
            full_df.ix[row,'Hardwood'] = 1

In [12]:
# dummy variable for laundry value in 'features' column
full_df['Laundry'] = 0
laundry_list = ['Laundry','In Unit Laundry', 'laundry']
for row in range(0,len(full_df)):
    for feature in full_df.iloc[row]['features']:               
        if feature in laundry_list:
            full_df.ix[row,'Laundry'] = 1

In [13]:
# dummy variable for dishwasher value in 'features' column
full_df['Dishwasher'] = 0
dish_list = ['Dishwasher', 'dishwasher']
for row in range(0,len(full_df)):
    for feature in full_df.iloc[row]['features']:               
        if feature in dish_list:
            full_df.ix[row,'Dishwasher'] = 1

In [14]:
full_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124011 entries, 0 to 124010
Data columns (total 29 columns):
bathrooms            124011 non-null float64
bedrooms             124011 non-null int64
building_id          124011 non-null object
created              124011 non-null datetime64[ns]
description          124011 non-null object
display_address      124011 non-null object
features             124011 non-null object
interest_level       49352 non-null object
latitude             124011 non-null float64
listing_id           124011 non-null int64
longitude            124011 non-null float64
manager_id           124011 non-null object
photos               124011 non-null object
price                124011 non-null int64
street_address       124011 non-null object
year                 124011 non-null int64
month                124011 non-null int64
day                  124011 non-null int64
hour                 124011 non-null int64
weekday              124011 non-null int64
photos_

## listings by date

In [15]:
full_df.created.min(), full_df.created.max()

(Timestamp('2016-04-01 22:12:41'), Timestamp('2016-06-29 21:55:35'))

In [16]:
grouped_by_date_df = full_df.groupby('created')['listing_id'].count().reset_index().copy()
grouped_by_date_df.columns = ['created_date','count_of_listings']
grouped_by_date_df.head(5)

,created_date,count_of_listings
0,2016-04-01 22:12:41,1
1,2016-04-01 22:23:31,1
2,2016-04-01 22:50:13,1
3,2016-04-01 22:54:42,1
4,2016-04-01 22:56:00,1


## listings by day of month

In [17]:
len(full_df.listing_id.unique())

124011

In [18]:
grouped_by_day_df = full_df.groupby('day')['listing_id'].count().reset_index().copy()
grouped_by_day_df.columns = ['day_of_month','count_of_listings']
grouped_by_day_df.head(3)

,day_of_month,count_of_listings
0,1,2279
1,2,4401
2,3,4406


In [19]:
grouped_by_day_df.to_csv('grouped_by_day_df.csv',index=False)

## listings by day of of week

In [20]:
grouped_by_dow_df = full_df.groupby('weekday')['listing_id'].count().reset_index().copy()
grouped_by_dow_df.columns = ['day_of_week','count_of_listings']
grouped_by_dow_df['day_of_week'] = grouped_by_dow_df['day_of_week']\
        .map( {0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday', 5: 'Sautrday', 6: 'Sunday'}).astype(str)
grouped_by_dow_df.head(7)

,day_of_week,count_of_listings
0,Monday,10462
1,Tuesday,20938
2,Wednesday,23693
3,Thursday,20684
4,Friday,19134
5,Sautrday,17679
6,Sunday,11421


In [21]:
grouped_by_dow_df.to_csv('grouped_by_day_of_week_df.csv',index=False)

## listings by number of bedrooms

In [22]:
grouped_by_bedrooms_df = full_df.groupby('bedrooms')['listing_id'].count().reset_index().copy()
grouped_by_bedrooms_df.columns = ['bedrooms','count_of_listings']
grouped_by_bedrooms_df

,bedrooms,count_of_listings
0,0,23564
1,1,39608
2,2,37114
3,3,18149
4,4,4887
5,5,569
6,6,112
7,7,6
8,8,2


In [23]:
grouped_by_bedrooms_df.to_csv('grouped_by_num_of_bedrooms_df.csv',index=False)

## gps coords for listings

In [24]:
gps_df = full_df[['latitude','longitude']].copy()
gps_df.head(5)

,latitude,longitude
0,40.71,-73.94
1,40.79,-73.97
2,40.74,-74.00
3,40.75,-73.97
4,40.82,-73.95


In [25]:
gps_df.to_csv('gps_coords_df.csv',index=False)

## gps coords for listings w/ interest level

In [26]:
gps__interestlevel_df = train_df[['latitude','longitude','interest_level']].copy()
gps__interestlevel_df.head(5)

,latitude,longitude,interest_level
10,40.71,-73.94,medium
10000,40.79,-73.97,low
100004,40.74,-74.00,high
100007,40.75,-73.97,low
100013,40.82,-73.95,low


In [27]:
gps__interestlevel_df.to_csv('gps_coords_w_interestlevel_df.csv',index=False)

# interest level

In [28]:
interestlevel_df = train_df.groupby('interest_level')['listing_id'].count().reset_index().copy()
interestlevel_df.columns = ['interest_level','count_of_listings']
interestlevel_df = interestlevel_df.sort_values(by='count_of_listings', ascending=False)
interestlevel_df.head()

,interest_level,count_of_listings
1,low,34284
2,medium,11229
0,high,3839


In [29]:
interestlevel_df.to_csv('interest_level_df.csv',index=False)

## time series by day

In [30]:
full_df['ymd'] = full_df['created'].map(lambda x: x.strftime('%Y-%m-%d'))

In [31]:
grouped_by_date_df = full_df.groupby('ymd')['listing_id'].count().reset_index().copy()
grouped_by_date_df.columns = ['date','count_of_listings']
grouped_by_date_df.head(3)

,date,count_of_listings
0,2016-04-01,8
1,2016-04-02,1655
2,2016-04-03,743


In [32]:
grouped_by_date_df.to_csv('interest_level_by_date_df.csv',index=False)

## Price of listing based on bedrooms and bathrooms

In [33]:
columns = ['bedrooms', '1 bathroom', '2 bathrooms', '3 bathrooms']
bed_df = pd.DataFrame( columns=columns)
bed_df = bed_df.fillna(0)
print(bed_df)

Empty DataFrame
Columns: [bedrooms, 1 bathroom, 2 bathrooms, 3 bathrooms]
Index: []


In [34]:
beds = 1
for i in range(3):
    bed_df.set_value(i, 'bedrooms', beds)
    beds +=1

print(bed_df)

  bedrooms 1 bathroom 2 bathrooms 3 bathrooms
0        1        NaN         NaN         NaN
1        2        NaN         NaN         NaN
2        3        NaN         NaN         NaN


In [35]:
mean_bed_df = full_df.groupby(['bedrooms', 'bathrooms'])['price'].mean().reset_index().copy()
mean_bed_df

,bedrooms,bathrooms,price
0,0,0.00,3506.32
1,0,1.00,2488.90
2,0,1.50,3295.45
3,0,2.00,5486.70
4,0,3.00,7383.33
5,0,4.00,7995.00
6,1,0.00,2864.94
7,1,1.00,3189.51
8,1,1.50,4504.57
9,1,2.00,4609.69


In [36]:
bed_df.set_value(0, '1 bathroom', 3189.51)
bed_df.set_value(0, '2 bathrooms', 4609.69)
bed_df.set_value(0, '3 bathrooms', 3806.90)
bed_df.set_value(1, '1 bathroom', 3467.23)
bed_df.set_value(1, '2 bathrooms', 5499.13)
bed_df.set_value(1, '3 bathrooms', 9096.83)
bed_df.set_value(2, '1 bathroom', 3910.16)
bed_df.set_value(2, '2 bathrooms', 5280.48)
bed_df.set_value(2, '3 bathrooms', 10778.07)

,bedrooms,1 bathroom,2 bathrooms,3 bathrooms
0,1,3189.51,4609.69,3806.90
1,2,3467.23,5499.13,9096.83
2,3,3910.16,5280.48,10778.07


In [37]:
bed_df.to_csv('bed_df.csv',index=False)

## count of listings for each selected feature

In [38]:
columns = ['feature','including', 'excluding']
feature_df = pd.DataFrame(columns=columns)
feature_df = feature_df.fillna(0)

In [39]:
list_including = []
list_excluding = []
def group(list):
    for i in list:
        grouped_by_f_df = full_df.groupby([i])['listing_id'].count().reset_index().copy()
        list_including.append(grouped_by_f_df['listing_id'][0])
        list_excluding.append(grouped_by_f_df['listing_id'][1])      
list = ["Elevator", "Laundry", "Doorman", "Hardwood", "Dishwasher"]
group(list)

print (list_including)
print (list_excluding)



[58184, 123932, 71509, 123665, 71988]
[65827, 79, 52502, 346, 52023]


In [40]:
feature_df['including'] = list_including
feature_df['excluding'] = list_excluding
feature_df['feature'] = ["Elevator", "Laundry", "Doorman", "Hardwood", "Dishwasher"]
feature_df

,feature,including,excluding
0,Elevator,58184,65827
1,Laundry,123932,79
2,Doorman,71509,52502
3,Hardwood,123665,346
4,Dishwasher,71988,52023


In [41]:
feature_df.to_csv('feature_df.csv',index=False)

## price of listing based on available feature

In [47]:
columns = ['feature','price_no',"price_yes"]
fprice_df = pd.DataFrame(columns=columns)
fprice_df = fprice_df.fillna(0)
print(fprice_df)

Empty DataFrame
Columns: [feature, price_no, price_yes]
Index: []


In [48]:
list1=['Elevator','Laundry','Doorman','Hardwood',"Dishwasher"]

In [49]:
list2 = []
list3 =[]
def group(list):
    for i in list:
        grouped_by_fprice_df = full_df.groupby([i])['price'].mean().reset_index().copy()
        list2.append(grouped_by_fprice_df['price'][0])
        list3.append(grouped_by_fprice_df['price'][1])
list = ["Elevator", "Laundry", "Doorman", "Hardwood", "Dishwasher"]
group(list)
print (list2)
print (list3)


[3277.6561769558643, 3781.7866087854632, 3239.8919296871723, 3785.1972506368011, 3512.3598516419402]
[4226.5125860209337, 3055.8607594936707, 4518.7680278846519, 2397.0317919075146, 4153.5096015224035]


In [50]:
fprice_df['feature'] = list1
fprice_df['price_no'] = list2
fprice_df['price_yes'] = list3
print(fprice_df)

      feature  price_no  price_yes
0    Elevator   3277.66    4226.51
1     Laundry   3781.79    3055.86
2     Doorman   3239.89    4518.77
3    Hardwood   3785.20    2397.03
4  Dishwasher   3512.36    4153.51


In [51]:
fprice_df.to_csv('fprice.csv',index=False)